## Avaliação Continuada (AC2) - Árvore de Decisão

Nesse notebook, iremos ver como aplicar o método de árvore de decisão na nossa base de dados que foi processada no notebook `train_data.ipynb`.

Para começar então, vamos importar o notebook `train_data.ipynb` que irá conter tudo que precisamos.

In [1]:
import import_ipynb
import train_data

importing Jupyter notebook from train_data.ipynb


,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


,Category,Message
count,5572,5572
unique,2,5157
top,ham,"Sorry, I'll call later"
freq,4825,30


Shape antes de remover os dados.
(5572, 2)

Shape após remover os dados.
(5157, 2)

Quantidade de itens removidos: 415
Um dos textos antes do processamento:


'I sent you  &lt;#&gt;  bucks'

O mesmo texto, só que agora processado:


'sent you bucks'

Algumas palavras que iremos usar para montar o nosso vocabulário:
['go', 'until', 'jurong', 'point', 'crazy', 'available', 'only', 'in', 'bugis', 'great', 'world', 'la', 'buffet', 'cine', 'there', 'got', 'amore', 'wat']
Tamanho do Vocabulario:  3632
[ 1.39969841e-01 -1.17486238e-01 -1.10714950e-01  2.05853507e-01
  3.41195613e-01 -3.51610363e-01  1.71089068e-01  6.30936682e-01
 -2.57613868e-01  2.52222419e-01 -2.52344221e-01 -3.61966491e-01
  1.49087831e-01  2.60720700e-01 -4.47315834e-02 -2.46243462e-01
 -1.15892880e-01  2.68316333e-04  1.64664537e-02 -5.39395273e-01
  2.07198560e-01 -2.55643785e-01 -1.24959648e-01 -1.37009293e-01
  9.96919572e-02 -2.70122647e-01  1.30453985e-02 -2.06147701e-01
 -3.52618217e-01  1.66222602e-01  3.97628814e-01 -6.11065701e-02
  1.64686158e-01 -3.91479358e-02 -7.73691237e-02  2.91181833e-01
  2.49674052e-01 -1.94237232e-01 -8.70821849e-02 -4.84320670e-01
 -3.16953808e-01 -9.99857187e-02  3.48575637e-02  1.01693183e-01
  4.99485046e-01 -1.94202736e-01 -2

## Treinamento

Agora, com os dados separados, vamos começar o treinamento usando o método de Árvore de Decisão.

Primeiro, importamos o método que iremos usar.

In [2]:
import sklearn
from sklearn.tree import DecisionTreeClassifier

Depois, criamos o nosso modelo da seguinte forma:

In [3]:
decisionTree = DecisionTreeClassifier(criterion = "entropy", random_state=0)

E ai, já podemos começar o treinamento do modelo.

In [4]:
decisionTree.fit(train_data.x_train, train_data.y_train) 

DecisionTreeClassifier(criterion='entropy', random_state=0)

E com o modelo treinado, podemos começar a ver as métricas de acurácia do nosso modelo tentando predizer os valores de teste que separamos na etapa anterior.

In [5]:
y_pred = decisionTree.predict(train_data.x_test) 

Com os valores que foram classificados acima, podemos ver qual foi a sua performance usando o `sklearn` da seguinte forma:

In [6]:
results = sklearn.metrics.classification_report(train_data.y_test, y_pred, target_names=['ham', 'spam'])

print(results)

              precision    recall  f1-score   support

         ham       0.96      0.95      0.95      1352
        spam       0.67      0.73      0.70       196

    accuracy                           0.92      1548
   macro avg       0.81      0.84      0.83      1548
weighted avg       0.92      0.92      0.92      1548



Com a tabela acima, podemos tirar algumas conclusões como: 

A precisão para identificar mensagens verdadeiras é maior do que a precisão do que identificar mensagens de spam. E isso não necessariamente é ruim porque é melhor errar mais mensagens de spam do que mensagens verdadeiras.

Dessa forma, no pior dos casos ainda vemos o spam e ignoramos mas com a mensagem real indo parar no spam, é mais dificil de ir lá olhar.

E além disso, podemos checar a nossa acurácia com mais precisão tanto dos valores de teste quanto dos valores de treino da seguinte forma:

In [7]:
accuracyTest = train_data.accuracy_score(train_data.y_test, decisionTree.predict(train_data.x_test))
accuracyTrain = train_data.accuracy_score(train_data.y_train, decisionTree.predict(train_data.x_train))

print("Acurácia sobre os valores de teste: ", accuracyTest)
print("Acurácia sobre os valores de treino: ", accuracyTrain)

Acurácia sobre os valores de teste:  0.9198966408268734
Acurácia sobre os valores de treino:  1.0


E aqui vemos, temos, assim como mostrado no relatório anterior, certa de 91% de acurácia sobre os valores de teste e 100% para os valores de treino.

## Cross-Validation

Apesar de eu não entender tão bem como funciona a validação cruzada, eu quis trazer nesses métodos para ter um pequena noção de como a ordem dos dados de treinamento influenciam o nosso modelo.

Dessa forma, o cross-validation permite que a gente separe em partições os nossos dados, e eles por consequencia, serão usados para treinar várias vezes o modelo e assim podemos obter as métrica de acurácia de cada vez que foi treinado o modelo.

Com essas métricas, podemos verificar se a mudança na ordem dos dados tem uma mudança significativa na acurácia do nosso modelo.

Vamos começar então com a importação das bibliotecas que iremos usar.

In [8]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import ShuffleSplit

Depois, vamos criar algumas variáveis para configurarmos como será a cross-validation.

In [9]:
n_splits = 5 # o número de partições
test_size = 0.3 # a porcentagem que será usada para teste, o restante é usada no treino
cv = ShuffleSplit(n_splits=n_splits, test_size=test_size, random_state=0) # define a função que irá realizar o rearranjo dos dados de treino e como será divido os dados.

E assim, podemos agora chamar a função `cross_val_score` que irá nos retornar a acurácia, assim como, o desvio da acurácia que nos diz quanto a ordem os dados está influenciando o modelo.

In [10]:
scores = cross_val_score(decisionTree, train_data.x, train_data.y, cv=cv)

print("%0.2f acurácia com o desvio de %0.2f" % (scores.mean(), scores.std()))

0.93 acurácia com o desvio de 0.01


E podemos ver que a acurácia teve uma ligeira melhoria, de 91% para 93%.

O interessante disso é ver que o modelo Árvore de Decisão não foi tão influenciado pela ordenação dos dados, diferente do Perceptron por exemplo.